In [ ]:
import json 
f = open('report-val.json')
overall_stats = f.readline()

In [12]:
import numpy as np

USE_VAL = False
NUM_RECORDS = None

if USE_VAL:
    data = np.loadtxt('val-data.csv', delimiter=',').astype(float)
    labels = np.loadtxt('val-labels.csv', delimiter=',')
    NUM_RECORDS = 2500
else:
    data = np.concatenate((np.loadtxt('train-data1.csv', delimiter=','), 
                            np.loadtxt('train-data2.csv', delimiter=','))).astype(float)
    print(data.shape)
    labels = np.loadtxt('train-labels.csv', delimiter=',')[:60000]
    print(labels.shape)
    NUM_RECORDS = 60000

(60000, 96)
(60000,)


# Explore different ways to canonicalize the I/O
 1. Data is coming in as (2500, 78) -> first reshape to (2500, 6, 13)
 2. Sort rows based on a column value
 3. reshape back to (2500, 78)

In [14]:
from sklearn.linear_model import LogisticRegression

io_data = data[:, 18:]
reshaped_data = io_data.reshape((NUM_RECORDS, 6, 13))

val1, train1 = [], []
VAL_CUTOFF = int(NUM_RECORDS * 0.8)

for i in range(13):  
    fresh_data = np.zeros((NUM_RECORDS, 78))
    for t in range(NUM_RECORDS): # t for trial
        rows = reshaped_data[t]
        sorted_rows = rows[rows[:,i].argsort()]
        fresh_data[t] = sorted_rows.reshape(78)
    
    #print(data[:, :18].shape)
    #print(fresh_data.shape)
    reformatted_data = np.concatenate((data[:, :18], fresh_data), axis=1)
    for c in range(96):
        reformatted_data /= np.max(np.abs(reformatted_data[:,c])) + 1e-4
        reformatted_data[:,c] -= np.mean(reformatted_data[:,c])
    
    log_reg = LogisticRegression()
    log_reg.fit(reformatted_data[:VAL_CUTOFF], labels[:VAL_CUTOFF])
    train1.append(log_reg.score(reformatted_data[:VAL_CUTOFF], labels[:VAL_CUTOFF]))
    

    val1.append(log_reg.score(reformatted_data[VAL_CUTOFF:], labels[VAL_CUTOFF:]))

print(train1)
print("**********************")
print(val1)


[0.79643750000000002, 0.79622916666666665, 0.79597916666666668, 0.7963958333333333, 0.79668749999999999, 0.79693749999999997, 0.79658333333333331, 0.80189583333333336, 0.79785416666666664, 0.79610416666666661, 0.79549999999999998, 0.79956249999999995, 0.79777083333333332]
**********************
[0.79208333333333336, 0.79108333333333336, 0.79183333333333328, 0.79100000000000004, 0.79091666666666671, 0.79116666666666668, 0.79216666666666669, 0.79458333333333331, 0.79033333333333333, 0.79025000000000001, 0.78983333333333339, 0.79300000000000004, 0.79574999999999996]


In [ ]:
# logistic regression isn't super impressive ~82% accuracy. 
# let's try basic neural network instead

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

model = Sequential()

model.add(Dense(100, activation='relu', input_shape=(96,)))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

io_data = data[:, 18:]
reshaped_data = io_data.reshape((2500, 6, 13))

# val1, val2, train1, train2 = [], [], [], []

for i in range(13):  
    fresh_data = np.zeros((2500, 78))
    for t in range(2500): # t for trial
        rows = reshaped_data[t]
        sorted_rows = rows[rows[:,i].argsort()]
        fresh_data[t] = sorted_rows.reshape(78)
    
    #print(data[:, :18].shape)
    #print(fresh_data.shape)
    reformatted_data = np.concatenate((data[:, :18], fresh_data), axis=1)
    for c in range(96):
        reformatted_data /= np.max(np.abs(reformatted_data[:,c])) + 1e-4
        reformatted_data[:,c] -= np.mean(reformatted_data[:,c])
       

    model.fit(reformatted_data[:2000], labels[:2000], 
             batch_size=16, nb_epoch=5, verbose=1)
    print(model.evaluate(reformatted_data[2000:], labels[2000:], verbose=1))

In [ ]:
lin_model.get_params()